# Extraindo dados da Covid19 via API
### Fonte de Dados: https://brasil.io/dataset/covid19/
### Augusto SPINELLI

### Licença
Os dados dados convertidos estão sob a licença Creative Commons Attribution ShareAlike. Caso utilize os dados, cite a fonte original e quem tratou os dados, como: Fonte: Secretarias de Saúde das Unidades Federativas, dados tratados por Álvaro Justen e colaboradores/Brasil.IO. Caso compartilhe os dados, utilize a mesma licença.

In [1]:
#importando dependencias
import pandas as pd
import numpy as np
import os

#Caminho dos arquivos RAW
raw_data_path = os.path.join(os.path.pardir, 'data', 'raw')
data_path = os.path.join(raw_data_path, 'ceps2.csv')

#criando dataframes brutos/raw
raw_df = pd.read_csv(data_path)

#renomeando colunas com encoding errado
df = raw_df.rename(columns={"evolu��o": "evolucao", "dt_�bito": "dt_obito"})

#mapeando data de atualizacao de forma correta
def GetData(dataFull):
    d = dataFull.split(';')[0]
    return d

df['dt_registro'] = df['Data_atualiza��o;OBJECTID'].map(lambda x: GetData(x))

#removendo coluna inútil
del df['Data_atualiza��o;OBJECTID']

#vendo info do dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50640 entries, 0 to 50639
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   dt_notific             50613 non-null  object 
 1   dt_inicio_sintomas     50603 non-null  object 
 2   bairro_resid__estadia  50640 non-null  object 
 3   ap_residencia_estadia  50640 non-null  float64
 4   evolucao               50640 non-null  object 
 5   dt_obito               4727 non-null   object 
 6   CEP                    50640 non-null  int64  
 7   dt_registro            50640 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 3.1+ MB


## Uso do Mapa com Leaflet

O Leaflet é uma biblioteca JavaScript de código aberto amplamente usada para criar aplicativos de mapeamento da web

In [40]:
#testando ipyleaflet
from ipyleaflet import Map, Marker

center = (-22.9980967, -43.2654989)

m = Map(center=center, zoom=13)

m.add_layer(Marker(location=center))

display(m)


Map(center=[-22.9980967, -43.2654989], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_tit…

## Usando uma API de CEPs Brasileiros para buscar mais dados

### CEP Aberto
É um projeto que visa prover acesso gratuito e construir de maneira colaborativa uma base de dados com os Codigos de Enderecamento Postal (CEP) geolocalizados de todo o Brasil. Colabore melhorando a informação de CEPs que você conhece. Registre-se gratuitamente para ter acesso a API e poder baixar a base de dados. O CEP Aberto possui informação de 1137131 CEPs distribuídos em 10659 cidades e municípios

In [6]:
#usando a api do cep aberto para pesquisar ceps
import requests

token = "##SECRET##" #pegue seu token pessoal após cadastro no CEP Aberto
headers = {'Authorization': 'Token token=%s' % token}


def search_by_cep(cep):
    url = "https://www.cepaberto.com/api/v3/cep?cep=" + cep
    response = requests.get(url, headers=headers)
    return response.json()

In [45]:
#testando acesso a API
search_by_cep("22041001")


{'altitude': 23.0,
 'cep': '22041001',
 'latitude': '-22.9699083',
 'longitude': '-43.182299',
 'logradouro': 'Avenida Atlântica',
 'bairro': 'Copacabana',
 'complemento': '- de 2174 a 2634 - lado par',
 'cidade': {'ddd': 21, 'ibge': '3304557', 'nome': 'Rio de Janeiro'},
 'estado': {'sigla': 'RJ'}}

## Salvando dados de CEP em dicionario local

### Depois usaremos uma função lambda para mapear esses dados em nosso dataframe

In [46]:
import json
import os

raw_data_path = os.path.join(os.path.pardir, 'data', 'raw')
data_path = os.path.join(raw_data_path, 'dictCeps.json')
ceps = df.CEP.unique()
mydict = {}
str_cep = ""

In [ ]:
for cep in ceps:
    str_cep = str(cep)
    mydict[str_cep] = search_by_cep(str_cep)

with open(data_path, 'w') as fp:
    json.dump(mydict, fp)